In [ ]:
!pip install optuna
!pip install optuna_integration
!pip install scikeras


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 3.4 MB/s eta 0:00:00


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import copy
import os
import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.pruners import HyperbandPruner
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split,StratifiedKFold, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import mutual_info_regression

df_train = pd.read_csv("/content/drive/MyDrive/train.csv")
df_train_ex=pd.read_csv("/content/drive/MyDrive/training_extra.csv")
df_test = pd.read_csv("/content/drive/MyDrive/test.csv")
df_train=pd.concat([df_train, df_train_ex], axis=0)
#df_train=df_train.reset_index(drop=True)
df_train.index = df_train['id']
df_train.index.name = None
#df_train.rename(columns={df_train.columns[0]: ""}, inplace=True)
print(df_train)
#df_train.reset_index(drop=True, inplace=True)  # Reset the index
#df_train.set_index("id", inplace=True)
#df_train["id"] = df_train.index
X_train = df_train.drop(columns=['id','Price'])
y_train = df_train['Price']
print(X_train)
print(y_train)
X_test= copy.deepcopy(df_test)
X_test.drop(columns=['id'],inplace=True)

print(X_train.dtypes)
print(X_train.isnull().sum())
print(X_test.isnull().sum())

X_train['Compartments'].fillna(X_train['Compartments'].median(), inplace=True)
X_train['Weight Capacity (kg)'].fillna(X_train['Weight Capacity (kg)'].median(), inplace=True)

X_test['Compartments'].fillna(X_test['Compartments'].median(), inplace=True)
X_test['Weight Capacity (kg)'].fillna(X_train['Weight Capacity (kg)'].median(), inplace=True)
# def impute_categorical_uniform(df, columns):
#     """
#     Imputes missing values in a categorical column using a uniform distribution.

#     Args:
#     df (pd.DataFrame): Input dataframe.
#     columns: List of column names to impute.

#     Returns:
#     pd.DataFrame: Dataframe with imputed values.
#     """
#     df_copy = df.copy()
#     for column in columns:
#         # Find unique categories (excluding NaN)

#         # Find unique categories (excluding NaN)
#         #unique_categories = df[column].dropna().unique()

#     # Count missing values
#         num_missing = df[column].isna().sum()

#     # If there are missing values, sample from the unique categories
#         if num_missing > 0:


#                 # Sample based on observed category probabilities
#             category_probs = df_copy[column].value_counts(normalize=True)
#             imputed_values = np.random.choice(category_probs.index, size=num_missing, p=category_probs.values)
#             df_copy.loc[df_copy[column].isna(), column] = imputed_values

#     return df_copy




# X_train=impute_categorical_uniform(X_train, columns=['Brand', 'Material', 'Laptop Compartment', 'Waterproof', 'Style', 'Color', 'Size'])
# X_test = impute_categorical_uniform (X_test, columns=['Brand', 'Material', 'Laptop Compartment', 'Waterproof', 'Style', 'Color', 'Size'])

obj_cols = X_test.select_dtypes(include=['object']).columns
X_train[obj_cols] = X_train[obj_cols].fillna('None')
X_test[obj_cols] = X_test[obj_cols].fillna('None')


def detect_outliers_iqr(df, columns):
    """
    Detects outliers using the IQR method.

    Args:
    df (pd.DataFrame): Input dataframe.
    columns (list): List of numerical columns to check.

    Returns:
    pd.DataFrame: DataFrame with outlier information.
    """
    df_outliers = df.copy()

    for col in columns:
        Q1 = df_outliers[col].quantile(0.25)
        Q3 = df_outliers[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Detect outliers
        outliers = df_outliers[(df_outliers[col] < lower_bound) | (df_outliers[col] > upper_bound)]
        print(f"{col}: {len(outliers)} outliers detected.")

    return df_outliers

# Detect outliers in numerical columns
numerical_cols = ['Compartments', 'Weight Capacity (kg)']

detect_outliers_iqr(X_train, numerical_cols)

detect_outliers_iqr(pd.DataFrame(y_train, columns=['Price']), ['Price'])
# Initialize Isolation Forest
# iso_forest = IsolationForest(contamination=0.05, random_state=42)

# # Fit on training data (excluding target variable)
# outliers = iso_forest.fit_predict(X_train)  # Returns 1 (normal) or -1 (outlier)

# # Remove detected outliers
# X_train_filtered = X_train[outliers == 1]
# y_train_filtered = y_train[outliers == 1]


scaler = StandardScaler()
columns_to_standardize = ['Compartments', 'Weight Capacity (kg)']
X_train[columns_to_standardize] = scaler.fit_transform(X_train[columns_to_standardize])
X_test[columns_to_standardize] = scaler.transform(X_test[columns_to_standardize])
#X_train.drop('Color',axis=1,inplace=True)


def target_encode_blended(X_train, y_train, X_test, cat_columns, alpha=10, n_splits=25):
    """
    Performs target encoding with blending without concatenating X_train and y_train.

    Args:
        X_train (pd.DataFrame): Training feature dataframe (without the target column).
        y_train (pd.Series): Target variable.
        cat_columns (list): List of categorical columns to encode.
        alpha (int): Smoothing factor (higher values make it closer to the global mean).
        n_splits (int): Number of splits for KFold cross-validation.

    Returns:
        pd.DataFrame: Transformed training dataset with encoded columns.
    """
    train_encoded = X_train.copy()
    test_encoded = X_test.copy()
    global_mean = y_train.mean()

    # Initialize new encoded columns
    for col in cat_columns:
        train_encoded[col + '_target_enc'] = np.nan

    # KFold cross-validation for encoding
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    for train_idx, val_idx in kf.split(X_train):
        fold_X_train, fold_X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        fold_y_train = y_train.iloc[train_idx]

        for col in cat_columns:
            # Compute smoothed mean for each category
            category_means = fold_y_train.groupby(fold_X_train[col]).agg(['mean', 'count'])
            category_means['smooth'] = (category_means['count'] * category_means['mean'] + alpha * global_mean) / (category_means['count'] + alpha)

            # Map smoothed values to the validation set
            train_encoded.loc[X_train.index[val_idx], col + '_target_enc'] = fold_X_val[col].map(category_means['smooth'])

    final_mapping = {}
    for col in cat_columns:
        category_means = y_train.groupby(X_train[col]).agg(['mean', 'count'])
        category_means['smooth'] = (category_means['count'] * category_means['mean'] + alpha * global_mean) / (category_means['count'] + alpha)

        final_mapping[col] = category_means['smooth']

        test_encoded[col + '_target_enc'] = X_test[col].map(final_mapping[col])


    return train_encoded,test_encoded


              id         Brand   Material    Size  Compartments  \
0              0      Jansport    Leather  Medium           7.0   
1              1      Jansport     Canvas   Small          10.0   
2              2  Under Armour    Leather   Small           2.0   
3              3          Nike      Nylon   Small           8.0   
4              4        Adidas     Canvas  Medium           1.0   
...          ...           ...        ...     ...           ...   
4194313  4194313          Nike     Canvas     NaN           3.0   
4194314  4194314          Puma    Leather   Small          10.0   
4194315  4194315      Jansport     Canvas   Large          10.0   
4194316  4194316          Puma     Canvas     NaN           2.0   
4194317  4194317  Under Armour  Polyester  Medium           2.0   

        Laptop Compartment Waterproof      Style  Color  Weight Capacity (kg)  \
0                      Yes         No       Tote  Black             11.611723   
1                      Yes       

<ipython-input-3-9e0e37ecd7fd>:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train['Compartments'].fillna(X_train['Compartments'].median(), inplace=True)
<ipython-input-3-9e0e37ecd7fd>:45: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(v

Compartments: 0 outliers detected.
Weight Capacity (kg): 0 outliers detected.
Price: 0 outliers detected.


In [ ]:
# @title Default title text

from tensorflow.keras import mixed_precision
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ReduceLROnPlateau

#mixed_precision.set_global_policy('mixed_float16')
print(tf.test.is_built_with_cuda())  # Should return True
print(tf.test.is_gpu_available())
print("Available GPUs:", tf.config.list_physical_devices('GPU'))
tf.config.optimizer.set_jit(False)


tf.keras.mixed_precision.set_global_policy('float32')
gpus = tf.config.list_physical_devices('GPU')
AUTOTUNE = tf.data.AUTOTUNE

if gpus:
    try:
        # Set TensorFlow to use only the first GPU
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("GPU enabled:", gpus[0])
    except RuntimeError as e:
        print(e)

#Define K for K-Fold Cross-Validation
K = 5  # Number of folds
kf = KFold(n_splits=K, shuffle=True, random_state=42)  # Ensure randomness

target_encode_columns = ['Brand', 'Material', 'Laptop Compartment', 'Waterproof', 'Style', 'Size', 'Color']

# def objective(trial):

    # Create fresh copies to avoid modifying original data
    #  X_train_local = copy.deepcopy(X_train)
    #  X_test_local = copy.deepcopy(X_test)
    #  y_train_local = copy.deepcopy(y_train)

    #  #alpha = trial.suggest_float("alpha", 10, 12,step=1)
    #  #learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)
    #  #l1_l2_reg = trial.suggest_loguniform("l1_l2_reg", 1e-5, 1e-2)
    #  #num_units_1 = trial.suggest_int("num_units_1", 64, 512, step=32)
    #  #num_units_2 = trial.suggest_int("num_units_2", 32, 256, step=16)
    #  #num_units_3= trial.suggest_int("num_units_3", 16, 128, step=16)
    #  #num_units_4= trial.suggest_int("num_units_4", 8, 64, step=16)
    #  #batch_size = trial.suggest_categorical("batch_size", [128, 256, 512])
    #  print(trial.params)
    #  X_train_local, X_test_local =target_encode_blended(X_train_local,y_train_local, X_test_local, target_encode_columns, alpha=20)
    #  # Drop original categorical columns (they are replaced with target encodings)
    #  X_train_local.drop(columns=target_encode_columns, inplace=True)
    #  X_test_local.drop(columns=target_encode_columns, inplace=True)



# Store RMSE for each fold
#      val_rmse_scores = []

# # Loop through K-folds
#      for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_local)):  # Assuming X_train, y_train are defined
#       print(f"\n Training Fold {fold+1}/{K}...")

#       X_train_sub, X_val = X_train_local.iloc[train_idx], X_train_local.iloc[val_idx]
#       y_train_sub, y_val = y_train_local.iloc[train_idx], y_train_local.iloc[val_idx]
#       train_dataset = tf.data.Dataset.from_tensor_slices((X_train_sub, y_train_sub))
#       train_dataset = train_dataset.batch(2048).prefetch(tf.data.AUTOTUNE)
#       val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(2048).prefetch(tf.data.AUTOTUNE)



#       model = keras.Sequential([
#          keras.layers.Input(shape=(X_train_sub.shape[1],)),
#         #  keras.layers.Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.005)),
#         #  keras.layers.Dropout(0.1),
#           #keras.layers.Dense(256,activation='relu',kernel_regularizer=regularizers.l2(0.005)),
#         #  keras.layers.Dropout(0.1),
#           keras.layers.Dense(128,activation='relu',kernel_regularizer=regularizers.l2(0.0001)),
#           #keras.layers.BatchNormalization(),
#           #keras.layers.Dropout(0.2),
#          keras.layers.Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.0001)),
#          #keras.layers.BatchNormalization(),
#          #keras.layers.Dropout(0.1),
#          keras.layers.Dense(64, activation='relu',kernel_regularizer=regularizers.l2(0.0001)),
#          #keras.layers.BatchNormalization(),
#          #keras.layers.Dropout(0.2),
#          keras.layers.Dense(1, activation='linear')
#                                                       ])  #

#       optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
#       model.compile(optimizer=optimizer, loss='mse', metrics=[keras.metrics.RootMeanSquaredError()])
#       early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
#       lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
# # pruning_callback = TFKerasPruningCallback(trial, "val_root_mean_squared_error")

#       history = model.fit(train_dataset,
#                         epochs=20,
#                         verbose=1, validation_data=val_dataset,callbacks=[lr_scheduler])

#     # Return validation RMSE as the score to minimize
#       val_rmse = history.history['val_root_mean_squared_error'][-1]  # Last epoch RMSE
#       val_rmse_scores.append(val_rmse)  # Store RMSE score
#       print(f"✅ Fold {fold+1} RMSE: {val_rmse:.4f}")


#       mean_rmse = np.mean(val_rmse_scores)
#       std_rmse = np.std(val_rmse_scores)
#       print(f"Cross-validation RMSE: {mean_rmse:.4f} ± {std_rmse:.4f}")

# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=1)

X_train, X_test =target_encode_blended(X_train,y_train, X_test, target_encode_columns, alpha=20)
X_train.drop(columns=target_encode_columns, inplace=True)
X_test.drop(columns=target_encode_columns, inplace=True)



final_model = keras.Sequential([
#          keras.layers.Input(shape=(X_train.shape[1],)),
#         #  keras.layers.Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.005)),
#         #  keras.layers.Dropout(0.1),
           #keras.layers.Dense(256,activation='relu'),
           #keras.layers.BatchNormalization(),
#         #  keras.layers.Dropout(0.1),
           keras.layers.Dense(128,kernel_regularizer=regularizers.l2(0.0001)),
            #eras.layers.BatchNormalization(),
            keras.layers.LeakyReLU(alpha=0.1),
#         #  keras.layers.BatchNormalization(),
           #keras.layers.Dropout(0.1),
          keras.layers.Dense(128,kernel_regularizer=regularizers.l2(0.0001)),
            keras.layers.LeakyReLU(alpha=0.1),


#          #keras.layers.BatchNormalization(),
          #keras.layers.Dropout(0.1),
          keras.layers.Dense(64,kernel_regularizer=regularizers.l2(0.0001)),
          keras.layers.LeakyReLU(alpha=0.1),
#          #keras.layers.BatchNormalization(),
          #keras.layers.Dropout(0.1),
          keras.layers.Dense(1, activation='linear')
                                                       ])



optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
final_model.compile(optimizer=optimizer, loss='mse', metrics=[keras.metrics.RootMeanSquaredError()])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, min_lr=1e-6)
full_train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(2048).prefetch(tf.data.AUTOTUNE)
history = final_model.fit(full_train_dataset, epochs=30, verbose=1,callbacks=[early_stopping, lr_scheduler])
y_pred=final_model.predict(X_test)

y_pred = pd.DataFrame(y_pred, columns=["prediction"])  # Ensure it has a column name

# Concatenate X_test["id"] with y_pred_df along axis=1 (column-wise)
y_pred = pd.concat([df_test[["id"]], y_pred], axis=1)

# Save to CSV
y_pred.to_csv("y_pred.csv", index=False)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True
True
Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU enabled: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Epoch 1/30
1951/1951 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 1929.0175 - root_mean_squared_error: 43.3467 - learning_rate: 1.0000e-04
Epoch 2/30


/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,root_mean_squared_error
  current = self.get_monitor_value(logs)


1951/1951 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - loss: 1516.9105 - root_mean_squared_error: 38.9472 - learning_rate: 1.0000e-04
Epoch 3/30
1951/1951 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 1516.9957 - root_mean_squared_error: 38.9483 - learning_rate: 1.0000e-04
Epoch 4/30
1951/1951 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - loss: 1517.0645 - root_mean_squared_error: 38.9492 - learning_rate: 1.0000e-04
Epoch 5/30
1951/1951 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 1517.0363 - root_mean_squared_error: 38.9489 - learning_rate: 1.0000e-04
Epoch 6/30
1951/1951 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 1516.7683 - root_mean_squared_error: 38.9454 - learning_rate: 5.0000e-05
Epoch 7/30
1951/1951 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - loss: 1516.7405 - root_mean_squared_error: 38.9451 - learning_rate: 5.0000e-05
Epoch 8/30
1951/1951 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - loss: 1516.7006 - root_mean_squared_error: 38.9445 - learning_rate: 5.0000e-05
Epoch 9/30
1951/1951 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - loss: 